In [ ]:
#Loading required libraries
import cv2
import numpy as np
from sklearn import mixture
from sklearn.cluster import KMeans
import math
from igraph import *


In [ ]:
#This Cell deals with all the calculations required for the Neighbourhood edges
#This is called only once at the time of intializing the graph 
def neighbours(x,y,num,size):  #returns array index of pixels in neighbourhood of pixel location(x,y)
    neighbour=[]
    val_x=[x-1,x,x+1]
    val_y=[y-1,y,y+1]
    pos_x=[]
    pos_y=[]
    for i in val_x: 
        if i<0 or i==size[0]:
            pass
        else:
            pos_x.append(i)
    for i in val_y: 
        if i<0 or i==size[1]:
            pass
        else:
            pos_y.append(i)
    if num==4:
        for i in pos_x:
            for j in pos_y:
                if (i==x and j==y) or (i!=x and j!=y):
                    pass
                else:
                    neighbour.append((i,j))
    if num==8:
        for i in pos_x:
            for j in pos_y:
                if i==x and j==y:
                    pass
                else:
                    neighbour.append((i,j))
    return(neighbour)


def Dist_Dist_C(N,img):   #Calculates Parameter for N-link Calculation
    a=N[0]
    b=N[1]
    num=N[2]
    Dist= (N[0][0]-N[1][0])**2 + (N[0][1]-N[1][1])**2    # Dist= Euclidean in pixel location space
                                                         # Beta=1/(2*dist_C/number of edges)
                                                         # Dist_C= Euclidean in pixel color space
    
    Dist_C=np.linalg.norm(img[N[0][0],N[0][1]]-img[N[1][0],N[1][1]])   
    return Dist,Dist_C

def N_Calc(X_unknown,pix):     #Main function for N-link Calculation:Finding neighbours
    N_edges=[]
    #print(X_unknown.shape)
    for i in range(X_unknown.shape[0]):   #Loop for finding neighbourhood for each pixel
        for j in range(X_unknown.shape[1]):
            neighbour=neighbours(i,j,neighbourhood,X_unknown.shape)
            for n in range(len(neighbour)):
                mapped_x=i+pix[1]
                mapped_y=j+pix[0]
                N_edges.append(((mapped_x,mapped_y),(neighbour[n][0]+pix[1],neighbour[n][1]+pix[0]),len(neighbour))) #source pixel,destination pixel,number of edges for source pixel
    weight_N=np.empty(len(N_edges))
    #print(img.shape)
    sum_Dist_C=0
    for i in range(len(N_edges)):
        sum_Dist_C+=(img[N_edges[i][0]]-img[N_edges[i][1]])**2 
    Beta=(np.linalg.norm(sum_Dist_C))/len(N_edges)
    Beta=1/(2*Beta)
    #print(Beta)
    for i in range(len(N_edges)):
            Dist,Dist_C=Dist_Dist_C(N_edges[i],img)
            temp=-Beta*(Dist_C**2)
            temp=gamma*math.exp(temp)
            weight_N[i]=temp/Dist            #N link edge weight allocatio
    N=[]#list of edges and weights together
    for i in range(len(N_edges)):
        N.append((N_edges[i][0],N_edges[i][1],weight_N[i]))     
    L=[]
    for i in range(len(N_edges)):
        L.append(weight_N[i])
    
    return(N) #Returns all the nighbourhood edges possible for each pixel along with their weights

In [ ]:
def Model_fit(X_fore,fix_back,X_un,fix_back_idx,idx_un):
    g_f = mixture.GaussianMixture(n_components=num_cluster).fit(X_fore)
    g_b = mixture.GaussianMixture(n_components=num_cluster).fit(fix_back)
    g_b_CI=g_b.fit_predict(X_un)
    g_f_CI=g_f.fit_predict(X_un) 
    #finding component index
    Comp_index=np.empty((h,w,2))#0-->Comp_Index for Foreground or unknown model 1-->for background model
    idx_u=np.asarray(idx_un)
    cnt=0
    for i,j in zip(idx_u[:,0],idx_u[:,1]):
        Comp_index[i,j,0]=g_f_CI[cnt]
        Comp_index[i,j,1]=g_b_CI[cnt]
        cnt+=1
    return(g_f,g_b,Comp_index)

In [ ]:
#This Cell has all the functions for calculating the Terminal Edges for the unknown pixel sets
#This is calculated in every iteration
def D_calc(X,model,j):     #Function for calculating th weigght for a given pixel basd on tee GMM Clustr it belongs to
    j=int(j)
    Gauss=model.weights_[j]*(gaussian(X, model.means_[j], model.covariances_[j]))
    if Gauss==0:
        Gauss=0.0000000000000001
    D=-np.log(Gauss)
    return D
def gaussian(X, mu, cov):   #Function to calculate the Gauassian Probability for a given pixel
    n = X.shape[0]
    diff = (X - mu).T
    g=1 / ((2 * np.pi) ** (n / 2) * np.linalg.det(cov) ** 0.5) * np.exp(-0.5 * np.dot(np.dot(diff.T, np.linalg.inv(cov)), diff))
    if g==0:
        g=0.0000000000000001
    return g
def T_Calc(g_f,g_b,comp_id,X_un,idx_un): #The main function to calculate the Terminal Edges
    #Allocation of t links
    #T-Edges
    T=[]
    idx_u=np.asarray(idx_un)
    

    cnt=0
    for i,j in zip(idx_u[:,0],idx_u[:,1]):
        D_fore=D_calc(X_un[cnt],g_f,comp_id[i,j,0])
        D_back=D_calc(X_un[cnt],g_b,comp_id[i,j,1])
        #print(D_fore,D_back)
        T.append(((i,j),Back,D_back))
        T.append((Fore,(i,j),D_fore))
        cnt+=1
        #print(cnt)
    return(T)    #Returns all T edges with their weights

In [ ]:
#This function deals with graph creation and performing min-cut by taking All the possible Edges and their weights
#It gives the Min Cut obtained as the output 
def Graph_Cut__(Edges,fix_back_idx):
    Vertices=[]
#     idx_u=np.asarray(idx_un)
#     x=list(idx_u[:,0])
#     y=list(idx_u[:,1])
    
#     for i,j in zip(x,y):
#         Vertices.append((i,j))
    for i in range(h):
            for j in range(w):
                if(i,j) in fix_back_idx: 
                    pass
                else:
                    Vertices.append((i,j))
    Vertices.append(Fore)
    Vertices.append(Back)
    g=Graph()
    for i in range(len(Vertices)):
        g.add_vertex(name=Vertices[i])
    #print(g,len(idx_un))
    V_dict={}
    vs = VertexSeq(g)
    for v in g.vs:
        V_dict[v["name"]]=v.index

    
    g=g.TupleList(Edges, directed=False, vertex_name_attr="name", edge_attrs="weight")
    es = EdgeSeq(g)
    #print(len(es))
 
    a=V_dict[(-1,-1)]
    b=V_dict[(h,w)]
    #print(a,b)
    min_cut=g.st_mincut(a,b,capacity="weight")
    #print(min_cut)
    #print(len(min_cut.partition[0]),len(min_cut.partition[1]),len(Vertices))
    #pos1=min_cut.partition[0].index(a)
    #pos2=min_cut.partition[1].index(b)
    #print(pos1,pos2)
    #Energy=min_cut.value
    #print(Energy) 
    iV_dict = {v: k for k, v in V_dict.items()}
    return(min_cut,iV_dict)

In [ ]:
#This function updates the Matte which contains the segmentation for the given image, after every iteration
def Update_Matte(min_cut,iV_dict,Matte):
    for i in min_cut.partition[0]:
        x_loc=iV_dict[i][0]
        y_loc=iV_dict[i][1]
        if(x_loc!=-1 and Matte[x_loc,y_loc]!=1): 
            Matte[x_loc,y_loc]=1
    for i in min_cut.partition[1]:
        x_loc=iV_dict[i][0]
        y_loc=iV_dict[i][1]
        if(x_loc!=h and Matte[x_loc,y_loc]!=0):
            Matte[x_loc,y_loc]=0
    #print(Matte.shape)
    return Matte

In [ ]:
#This is the main calling function for every iteration
def Grab_Cut_ITER(Matte,X_un,idx_un):
    X_fore=[]
    idx_fore=[]
    for i in range(h):
        for j in range(w):
            if Matte[i,j]==1:
                X_fore.append(img[i,j])
                idx_fore.append((i,j))

    foreground_model,background_model,Cluster_index=Model_fit(X_fore,fix_back,X_un,fix_back_idx,idx_un)
    T_links=T_Calc(foreground_model,background_model,Cluster_index,X_un,idx_un)
    Edges=[]
    Edges.extend(T_links)
    Edges.extend(N_links)
    min_cut,correspondences=Graph_Cut__(Edges,idx_un)
    Matte=Update_Matte(min_cut,correspondences,Matte)
    return(min_cut.value,Matte)

In [ ]:
#Main function 
if __name__ == "__main__":
    im_name='teddy'
    img=cv2.imread('./images/'+im_name+'.jpg') #Function to load image
    size=img.shape
    h=size[0]   #number of rows in img
    w=size[1]   #number of col in img
    Fore=(-1,-1) #Foreground Terminal 
    Back=(h,w)   #Backgground Terminal
    gamma=5    #Parameter to be multiplied with N link weeight
    num_cluster=5 #Number of GMM Cluster
    neighbourhood=8 #Neighbourhhood of Pixels;Either 4 or 8
    bb=open('./bb/'+im_name+'.txt') #(x,y,x+w,y+h) Bounding Box 
    f=bb.readlines()
    f='.'.join(str(x) for x in f)
    pix = [int(i) for i in f.split() if i.isdigit()] #Bounding Box coordinates
    bb_h=pix[3]-pix[1]   #Bounding Box Height 
    bb_w=pix[2]-pix[0]   #Bounding Box Width
    X_unknown=np.empty((bb_h,bb_w,3))  #Collection of unknown pixels i.e. pixels inside the bounding box
    Matte=np.empty((h,w))              #Binary Matrix to store Segmentation '0'-->Background '1'-->Foreground
  
    fix_back=[]
    fix_back_idx=[]
    X_un=[]
    idx_un=[]
    for i in range(pix[1],pix[3],1):  #Tentatively initializing all unknown pixels as foreground
        for j in range(pix[0],pix[2],1):  #pix[0]-->pix[2] along w pix[1]-->pix[3] along h 
            Matte[i,j]=1
            X_un.append(img[i,j])
            idx_un.append((i,j))
    for i in range(h):                  #Storing the sure background area
        for j in range(w): 
            if Matte[i,j]!=1:
                fix_back.append(img[i,j])
                fix_back_idx.append((i,j))
                Matte[i,j]=0
    #print(len(fix_back),len(X_un))
    img3=np.empty((h,w,3))
    for i in range(h):
        for j in range(w):
            if Matte[i,j]==0:
                img3[i,j]=[0,0,0]
            elif Matte[i,j]==1:
                img3[i,j]=img[i,j]
    cv2.imwrite('./V7/bb_'+im_name+'.jpg',img3)
    cnt=0
    for i in range(bb_h):
        for j in range(bb_w):
            X_unknown[i,j]=X_un[cnt]
            cnt+=1 
    
    N_links=N_Calc(X_unknown,pix)
    foreground_model,background_model,Cluster_index=Model_fit(X_un,fix_back,X_un,fix_back_idx,idx_un)
    T_links=T_Calc(foreground_model,background_model,Cluster_index,X_un,idx_un)
    Edges=[]
    Edges.extend(T_links)
    Edges.extend(N_links)
    min_cut,correspondences=Graph_Cut__(Edges,fix_back_idx)
    Matte=Update_Matte(min_cut,correspondences,Matte)
    img2=np.empty((h,w,3))
    for i in range(h):
        for j in range(w):
            if Matte[i,j]==0:
                img2[i,j]=[0,0,0]
            else:
                img2[i,j]=img[i,j]
    cv2.imwrite('./V7/'+im_name + '_out.jpg',img2)
    E=[]
    E.append(min_cut.value)
    for iteration in range(3):
        Energy,Matte=Grab_Cut_ITER(Matte,X_un,fix_back_idx)
        img2=np.empty((h,w,3))
        for i in range(h):
            for j in range(w):
                if Matte[i,j]==0:
                    img2[i,j]=[0,0,0]
                else:
                    img2[i,j]=img[i,j]
        cv2.imwrite('./V7/'+im_name + '_out_.jpg',img2)
        E.append(Energy)
    
